In [1]:
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch

D:\Anaconda\envs\wyc_gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\wyc_gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\Anaconda\envs\wyc_gpu\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
train_dataset = torchvision.datasets.FashionMNIST('data', train=True, transform=ToTensor(), download=True)
test_dataset = torchvision.datasets.FashionMNIST('data', train=True, transform=ToTensor(), download=True)

In [3]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
img, label = next(iter(train_dataloader))
img.shape

torch.Size([64, 1, 28, 28])

In [4]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(28*28, 900)
        self.linear_2 = nn.Linear(900, 256)
        self.linear_3 = nn.Linear(256, 128)
        self.linear_4 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    def forward(self, input):
        x = input.view(-1, 28*28)
        x = self.relu(self.linear_1(x))
        x = self.relu(self.linear_2(x))
        x = self.relu(self.linear_3(x))
        output = self.linear_4(x)
        return output

In [5]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
model = Model().to(device)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr = 0.001)

In [6]:
def train(dataloader, model, loss_fn, opt):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    train_loss, train_correct = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        with torch.no_grad():
            train_loss += loss
            train_correct += (y_pred.argmax(1)==y).type(torch.int32).sum().item()
    train_loss /= num_batch
    train_correct /= size
    return train_correct, train_loss

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    test_loss, test_correct = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        test_loss += loss
        test_correct += (y_pred.argmax(1)==y).type(torch.int32).sum().item()
    test_loss /= num_batch
    test_correct /= size
    return test_correct, test_loss

In [8]:
def fit(epoches, train_dataloader, test_dataloader, model, loss_fn, opt):
    train_loss = []
    train_correct = []
    test_loss = []
    test_correct = []
    for epoch in range(epoches):
        epoch_train_correct, epoch_train_loss = train(train_dataloader, model, loss_fn, opt)
        epoch_test_correct, epoch_test_loss = test(test_dataloader, model, loss_fn)
        train_loss.append(epoch_train_loss)
        train_correct.append(epoch_train_correct)
        test_loss.append(epoch_test_loss)
        test_correct.append(epoch_test_correct)
        template = ("epoch:{:2d}, train_loss:{:.5f}, train_correct:{:.2f}%, test_loss:{:.5f}, test_correct:{:.2f}%")
        print(template.format(epoch, epoch_train_loss, epoch_train_correct*100, epoch_test_loss, epoch_test_correct*100))

In [9]:
fit(epoches=100, train_dataloader=train_dataloader, test_dataloader=test_dataloader, model=model, loss_fn=loss_fn, opt=opt)

epoch: 0, train_loss:2.29253, train_correct:23.13%, test_loss:2.27933, test_correct:27.91%
epoch: 1, train_loss:2.26253, train_correct:28.26%, test_loss:2.24285, test_correct:28.26%
epoch: 2, train_loss:2.21243, train_correct:28.30%, test_loss:2.17300, test_correct:29.66%
epoch: 3, train_loss:2.10708, train_correct:37.89%, test_loss:2.02189, test_correct:43.75%
epoch: 4, train_loss:1.89049, train_correct:46.36%, test_loss:1.73842, test_correct:48.57%
epoch: 5, train_loss:1.58276, train_correct:52.90%, test_loss:1.43929, test_correct:55.83%
epoch: 6, train_loss:1.32864, train_correct:57.43%, test_loss:1.23320, test_correct:57.97%
epoch: 7, train_loss:1.16246, train_correct:60.21%, test_loss:1.10068, test_correct:60.45%
epoch: 8, train_loss:1.05272, train_correct:62.16%, test_loss:1.00942, test_correct:63.59%
epoch: 9, train_loss:0.97451, train_correct:64.12%, test_loss:0.94189, test_correct:64.53%
epoch:10, train_loss:0.91535, train_correct:65.77%, test_loss:0.89032, test_correct:67.20%

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 6.00 GiB total capacity; 5.16 GiB already allocated; 0 bytes free; 5.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF